<a href="https://colab.research.google.com/github/Yamada8412/Prog2kakushin/blob/main/work1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pandasで適当なcsvファイルを読み込んでデータ解析をする

In [2]:
import pandas as pd
df = pd.read_excel('パスワード強度判定1.xlsx')

In [ ]:
def calc_len(p):
  global srg

  if len(p) <= 3:
    return False
  else:
    # 45-2 67-4 78-7 89-11 
    return


def calc_sca(p):
  #違う種類の文字の散らばり具合を調べる関数
  p_list = list(p)

  up_al = list()
  un_al = list()
  num   = list()
  sym   = list()

  for i in range(len(p_list)):
    if p_list[i].isnumeric():
      num.append(i)
    elif p_list[i].isupper():
      up_al.append(i)
    elif (p_list[i].islower()):
      un_al.append(i)
    else:
      sym.append(i)
  
  for 

    


False
